In [1]:
%pip install langchain-aws

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION", "us-east-1")

In [4]:
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)


In [7]:
from langchain_aws import BedrockEmbeddings
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")


In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)


In [9]:
from langchain_aws import ChatBedrock
llm = ChatBedrock(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs={
        "temperature": 0,
        "max_tokens": 1000
    }
)

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:
{context}
""")

In [11]:
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [12]:
response = retrieval_chain.invoke({"input": "what is langsmith"})
print(response['answer'])

Based on the provided context, here are the key points about LangSmith:

1. LangSmith is a platform for building production-grade LLM (Large Language Model) applications.

2. It is framework-agnostic and can be used with or without LangChain's open-source frameworks (langchain and langgraph).

3. Key features of LangSmith include:
- Observability: Helps debug non-deterministic LLM applications by providing meaningful insights
- Evals (Evaluations): Allows testing and optimizing applications using evaluation datasets and metrics
- Prompt Engineering: Provides tools to help develop and refine prompts for LLM applications

4. LangSmith enables users to:
- Monitor and evaluate applications
- Create dashboards to view metrics like RPS, error rates, and costs
- Run evaluations using off-the-shelf evaluators
- Collect and analyze human feedback
- Manage prompt versions and collaborate

5. It supports application development across all stages - from prototyping to beta testing to production.

